In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/microsoft-malware-prediction/sample_submission.csv
/kaggle/input/microsoft-malware-prediction/train.csv
/kaggle/input/microsoft-malware-prediction/test.csv


In [2]:
#!pip install pyspark

#import pyspark.pandas as ps

#from sklearn.datasets import load_boston
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import gc
import time
gc.enable()
print(os.listdir("../input"))
from sklearn.impute import SimpleImputer

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier 

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

['microsoft-malware-prediction']


In [3]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'float16',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'float16',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'float16'
        }
#%%time

In [4]:
# mostra as células que você quer visualizar e seus tipos de dados
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = [c for c,v in dtypes.items() if v in numerics]
categorical_columns = [c for c,v in dtypes.items() if v not in numerics]

retained_columns = numerical_columns + categorical_columns

In [5]:
df_train = pd.read_csv('../input/microsoft-malware-prediction/train.csv', usecols = retained_columns, dtype=dtypes,low_memory=True)

gc.collect()

0

In [6]:
# retirando os espaços dos nomes das colunas
Coluna_name=[]
for i in df_train:
    Coluna_name.append(i.strip()) 
df_train.columns = Coluna_name 

df_train.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0.0,NaN,0.0,0,0,0.0,0.0,10.0,0.0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0.0,NaN,0.0,0,0,0.0,0.0,8.0,0.0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0.0,NaN,0.0,0,0,0.0,0.0,3.0,0.0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0.0,NaN,0.0,0,0,0.0,0.0,3.0,1.0
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0.0,0.0,0.0,0,0,0.0,0.0,1.0,1.0


In [7]:
# verificando a quantidade de linha e coluna
df_train.shape

(8921483, 83)

In [8]:
train_reduze = df_train.sample(frac = 0.50, random_state = 5)

train_reduze.shape

(4460742, 83)

CORRELAÇÃO

In [9]:
# Matriz de correlação em relação ao target
correlacao = train_reduze.corr()['HasDetections']
correlacao

IsBeta                                              -0.000077
RtpStateBitfield                                     0.041207
IsSxsPassiveMode                                    -0.034474
DefaultBrowsersIdentifier                           -0.001642
AVProductStatesIdentifier                            0.116853
AVProductsInstalled                                 -0.149120
AVProductsEnabled                                   -0.042206
HasTpm                                               0.005900
CountryIdentifier                                    0.007571
CityIdentifier                                      -0.002228
OrganizationIdentifier                               0.002512
GeoNameIdentifier                                    0.004095
LocaleEnglishNameIdentifier                          0.015217
OsBuild                                              0.025059
OsSuite                                             -0.020582
IsProtected                                          0.057179
AutoSamp

In [10]:
var = train_reduze.corr()['HasDetections'].abs()<=0.6
dic = var.to_dict() 

for i, v in dic.items():
    if v is True:
        print([i])

['IsBeta']
['RtpStateBitfield']
['IsSxsPassiveMode']
['DefaultBrowsersIdentifier']
['AVProductStatesIdentifier']
['AVProductsInstalled']
['AVProductsEnabled']
['HasTpm']
['CountryIdentifier']
['CityIdentifier']
['OrganizationIdentifier']
['GeoNameIdentifier']
['LocaleEnglishNameIdentifier']
['OsBuild']
['OsSuite']
['IsProtected']
['AutoSampleOptIn']
['SMode']
['IeVerIdentifier']
['Firewall']
['UacLuaenable']
['Census_OEMNameIdentifier']
['Census_OEMModelIdentifier']
['Census_ProcessorCoreCount']
['Census_ProcessorManufacturerIdentifier']
['Census_ProcessorModelIdentifier']
['Census_PrimaryDiskTotalCapacity']
['Census_SystemVolumeTotalCapacity']
['Census_HasOpticalDiskDrive']
['Census_TotalPhysicalRAM']
['Census_InternalPrimaryDiagonalDisplaySizeInInches']
['Census_InternalPrimaryDisplayResolutionHorizontal']
['Census_InternalPrimaryDisplayResolutionVertical']
['Census_InternalBatteryNumberOfCharges']
['Census_OSBuildNumber']
['Census_OSBuildRevision']
['Census_OSInstallLanguageIdentifi

EXCLUINDO COLUNAS

In [11]:
train_reduze.drop(columns=['IsBeta','RtpStateBitfield','IsSxsPassiveMode','DefaultBrowsersIdentifier','AVProductStatesIdentifier',
                       'AVProductsInstalled','AVProductsEnabled','HasTpm','CountryIdentifier','CityIdentifier',
                       'OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','OsBuild','OsSuite',
                       'IsProtected','AutoSampleOptIn','SMode','IeVerIdentifier','Firewall','UacLuaenable',
                       'Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorCoreCount',
                       'Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier','Census_PrimaryDiskTotalCapacity',
                       'Census_SystemVolumeTotalCapacity','Census_HasOpticalDiskDrive','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches',
                       'Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical',
                       'Census_InternalBatteryNumberOfCharges','Census_OSBuildNumber','Census_OSBuildRevision','Census_OSInstallLanguageIdentifier',
                       'Census_OSUILocaleIdentifier','Census_IsPortableOperatingSystem','Census_IsFlightingInternal','Census_IsFlightsDisabled',
                       'Census_ThresholdOptIn','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Census_IsSecureBootEnabled',
                       'Census_IsWIMBootEnabled','Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable',
                       'Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Wdft_RegionIdentifier','PuaMode','SmartScreen',
                       'Census_ProcessorClass','Census_PrimaryDiskTypeName','Census_ChassisTypeName','Census_PowerPlatformRoleName',
                       'Census_InternalBatteryType','OsBuildLab','MachineIdentifier'], axis = 1,inplace = True)

In [12]:
# verificando a quantidade de linha e coluna
train_reduze.shape

(4460742, 22)

In [13]:
train_reduze.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,SkuEdition,Census_MDC2FormFactor,...,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,HasDetections
6414404,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1478.0,windows10,x64,10.0.0.0,rs2,Pro,Desktop,...,amd64,rs2_release,Professional,PROFESSIONAL,IBSClean,UNKNOWN,IS_GENUINE,Volume:GVLK,Retail,1.0
618844,win8defender,1.1.14901.4,4.14.17639.18041,1.269.149.0,windows10,x64,10.0.0.0,rs3,Home,Notebook,...,amd64,rs3_release_svc_escrow,Core,CORE,Upgrade,UNKNOWN,IS_GENUINE,Retail,Retail,0.0
2605303,win8defender,1.1.15200.1,4.18.1807.18075,1.275.316.0,windows10,x64,10.0.0.0,rs4,Pro,Desktop,...,amd64,rs4_release,Professional,PROFESSIONAL,UUPUpgrade,FullAuto,IS_GENUINE,OEM:DM,Retail,1.0
5803051,win8defender,1.1.15100.1,4.18.1806.18062,1.273.483.0,windows10,x64,10.0.0.0,rs3,Pro,Notebook,...,amd64,rs3_release_svc_escrow,Professional,PROFESSIONAL,Update,UNKNOWN,INVALID_LICENSE,Retail,Retail,1.0
2940637,win8defender,1.1.15200.1,4.18.1807.18075,1.275.138.0,windows10,x64,10.0.0.0,rs4,Pro,Desktop,...,amd64,rs4_release,Professional,PROFESSIONAL,IBSClean,FullAuto,IS_GENUINE,Volume:MAK,Retail,1.0


DADOS NULOS

In [14]:
# dados faltantes
train_reduze.isna().sum()

ProductName                         0
EngineVersion                       0
AppVersion                          0
AvSigVersion                        0
Platform                            0
Processor                           0
OsVer                               0
OsPlatformSubRelease                0
SkuEdition                          0
Census_MDC2FormFactor               0
Census_DeviceFamily                 0
Census_OSVersion                    0
Census_OSArchitecture               0
Census_OSBranch                     0
Census_OSEdition                    0
Census_OSSkuName                    0
Census_OSInstallTypeName            0
Census_OSWUAutoUpdateOptionsName    0
Census_GenuineStateName             0
Census_ActivationChannel            0
Census_FlightRing                   0
HasDetections                       0
dtype: int64

In [15]:
# tipo dos dados
train_reduze.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4460742 entries, 6414404 to 4776232
Data columns (total 22 columns):
 #   Column                            Dtype   
---  ------                            -----   
 0   ProductName                       category
 1   EngineVersion                     category
 2   AppVersion                        category
 3   AvSigVersion                      category
 4   Platform                          category
 5   Processor                         category
 6   OsVer                             category
 7   OsPlatformSubRelease              category
 8   SkuEdition                        category
 9   Census_MDC2FormFactor             category
 10  Census_DeviceFamily               category
 11  Census_OSVersion                  category
 12  Census_OSArchitecture             category
 13  Census_OSBranch                   category
 14  Census_OSEdition                  category
 15  Census_OSSkuName                  category
 16  Census_OSIns

LINHAS DUPLICADAS

In [16]:
# linhas duplicadas
print(train_reduze.duplicated())

6414404    False
618844     False
2605303    False
5803051    False
2940637    False
           ...  
6129021    False
3885613    False
4427320    False
7236256     True
4776232    False
Length: 4460742, dtype: bool


In [17]:
# remover as repetições, mas deixando a primeira ocorrência

#df_train.drop_duplicates(keep='first', inplace=True)

VARIÁVEIS CATEGÓRICAS

In [18]:
obj = (train_reduze.dtypes == 'category') #quais colunas que são categóricas dentro do dataset?
object_cols = list(obj[obj].index)

object_cols

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'Platform',
 'Processor',
 'OsVer',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing']

In [19]:
encoder = OrdinalEncoder() #faça a troca das categorias por números

train_reduze[object_cols] = encoder.fit_transform(train_reduze[object_cols]) #ltere somente as colunas categóricas do meu dataset
train_reduze

,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,SkuEdition,Census_MDC2FormFactor,...,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing,HasDetections
6414404,4.0,61.0,58.0,7538.0,0.0,1.0,0.0,2.0,6.0,2.0,...,0.0,3.0,14.0,14.0,2.0,5.0,1.0,4.0,5.0,1.0
618844,4.0,58.0,39.0,6882.0,0.0,1.0,0.0,3.0,4.0,8.0,...,0.0,5.0,2.0,2.0,8.0,5.0,1.0,2.0,5.0,0.0
2605303,4.0,62.0,58.0,8084.0,0.0,1.0,0.0,4.0,6.0,2.0,...,0.0,7.0,14.0,14.0,6.0,2.0,1.0,0.0,5.0,1.0
5803051,4.0,61.0,52.0,7719.0,0.0,1.0,0.0,3.0,6.0,8.0,...,0.0,5.0,14.0,14.0,7.0,5.0,0.0,2.0,5.0,1.0
2940637,4.0,62.0,58.0,7915.0,0.0,1.0,0.0,4.0,6.0,2.0,...,0.0,7.0,14.0,14.0,2.0,2.0,1.0,5.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6129021,4.0,61.0,58.0,7813.0,0.0,1.0,0.0,4.0,4.0,2.0,...,0.0,7.0,2.0,2.0,5.0,2.0,0.0,2.0,5.0,1.0
3885613,4.0,62.0,58.0,8193.0,0.0,1.0,0.0,1.0,4.0,8.0,...,0.0,0.0,5.0,5.0,7.0,2.0,1.0,2.0,5.0,0.0
4427320,4.0,62.0,26.0,7600.0,0.0,1.0,0.0,4.0,4.0,8.0,...,0.0,7.0,5.0,5.0,2.0,5.0,1.0,2.0,5.0,1.0
7236256,4.0,61.0,58.0,7809.0,0.0,1.0,0.0,2.0,4.0,8.0,...,0.0,3.0,5.0,5.0,5.0,5.0,1.0,0.0,5.0,1.0


OUTLIERS

In [20]:
# removendo outliers
# https://www.adamsmith.haus/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python

'''z_scores = stats.zscore(df_train)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df_train = df_train[filtered_entries]'''

'z_scores = stats.zscore(df_train)\n\nabs_z_scores = np.abs(z_scores)\nfiltered_entries = (abs_z_scores < 3).all(axis=1)\ndf_train = df_train[filtered_entries]'

In [21]:
#df_train.shape

SPLIT

In [22]:
print(list(train_reduze.columns))

['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'Platform', 'Processor', 'OsVer', 'OsPlatformSubRelease', 'SkuEdition', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_OSVersion', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_FlightRing', 'HasDetections']


In [23]:
y = train_reduze['HasDetections']

X_features = ['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'Platform', 'Processor', 
              'OsVer', 'OsPlatformSubRelease', 'SkuEdition', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_OSVersion', 
              'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 
              'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_FlightRing']
X = train_reduze[X_features]

In [24]:
X.head()

,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,SkuEdition,Census_MDC2FormFactor,...,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
6414404,4.0,61.0,58.0,7538.0,0.0,1.0,0.0,2.0,6.0,2.0,...,231.0,0.0,3.0,14.0,14.0,2.0,5.0,1.0,4.0,5.0
618844,4.0,58.0,39.0,6882.0,0.0,1.0,0.0,3.0,4.0,8.0,...,294.0,0.0,5.0,2.0,2.0,8.0,5.0,1.0,2.0,5.0
2605303,4.0,62.0,58.0,8084.0,0.0,1.0,0.0,4.0,6.0,2.0,...,335.0,0.0,7.0,14.0,14.0,6.0,2.0,1.0,0.0,5.0
5803051,4.0,61.0,52.0,7719.0,0.0,1.0,0.0,3.0,6.0,8.0,...,294.0,0.0,5.0,14.0,14.0,7.0,5.0,0.0,2.0,5.0
2940637,4.0,62.0,58.0,7915.0,0.0,1.0,0.0,4.0,6.0,2.0,...,335.0,0.0,7.0,14.0,14.0,2.0,2.0,1.0,5.0,5.0


In [25]:
from sklearn.model_selection import train_test_split


trainX, valX, trainy, valy = train_test_split(X, y,
                                              test_size=0.3, 
                                              random_state=12,
                                              stratify = y)
valX.shape

(1338223, 21)

NORMALIZAÇÃO

In [26]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))

trainX = sc.fit_transform(trainX)

valX = sc.transform(valX)

Criando o modelo LGBM Classifier

In [27]:
import lightgbm as lgb

model_lgb = lgb.LGBMClassifier(max_depth = 2)

type(model_lgb)

lightgbm.sklearn.LGBMClassifier

In [28]:
model_lgb.fit(trainX, trainy)

LGBMClassifier(max_depth=2)

In [29]:
lgb_pred = model_lgb.predict(valX)
lgb_pred

array([1., 1., 1., ..., 1., 0., 1.], dtype=float16)

lightgbm - DART: Dropouts meet Multiple Additive Regression Trees

In [30]:
lgb_dart = lgb.LGBMClassifier(booster = 'dart')

In [31]:
lgb_dart.fit(trainX, trainy)

[LightGBM] [Warning] Unknown parameter: booster


LGBMClassifier(booster='dart')

In [32]:
dart_pred = lgb_dart.predict(valX)
dart_pred

array([1., 1., 1., ..., 1., 0., 1.], dtype=float16)

## MODELO Extreme Gradient Boosting

In [33]:
#Extreme Gradient Boosting

#xgb_clf = XGBClassifier()
#xgb_clf.fit(trainX, trainy)

In [34]:
#predicao = xgb_clf.predict(valX)
#predicao

In [35]:
'''print(classification_report(valy, predicao))
print("accuracy_score", accuracy_score(valy, predicao))
predictions_probas = xgb_clf.predict_proba(valy)
print("roc-auc score for the class 1, from target 'HasDetections' ", roc_auc_score(valy, predictions_probas[:,1]))'''


'print(classification_report(valy, predicao))\nprint("accuracy_score", accuracy_score(valy, predicao))\npredictions_probas = xgb_clf.predict_proba(valy)\nprint("roc-auc score for the class 1, from target \'HasDetections\' ", roc_auc_score(valy, predictions_probas[:,1]))'

## MODELO GradientBoostingClassifier

In [36]:
#gb_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 42)
#gb_clf.fit(trainX, trainy)

In [37]:
#predicao = gb_clf.predict(valX)
#predicao

ALGORITMO BASELINE

In [38]:
#import numpy as np
#previsoes_de_base = np.ones(540)
#acuracia = accuracy_score(test_y, previsoes_de_base) * 100
#print("A acurácia do algoritmo de baseline foi %.2f%%" % acuracia)

DADOS TESTE

In [39]:
df_test = pd.read_csv('../input/microsoft-malware-prediction/test.csv', dtype=dtypes,low_memory=True)

gc.collect()

0

In [40]:
# retirando os espaços dos nomes das colunas
Coluna_name=[]
for i in df_test:
    Coluna_name.append(i.strip()) 
df_test.columns = Coluna_name 

df_test.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,0000010489e3af074adeac69c53e555e,win8defender,1.1.15400.5,4.18.1810.5,1.281.501.0,0,7.0,0,NaN,53447.0,...,807.0,8554.0,1.0,NaN,0.0,0,0,0.0,0.0,7.0
1,00000176ac758d54827acd545b6315a5,win8defender,1.1.15400.4,4.18.1809.2,1.279.301.0,0,7.0,0,NaN,53447.0,...,554.0,33105.0,1.0,NaN,0.0,0,0,0.0,1.0,12.0
2,0000019dcefc128c2d4387c1273dae1d,win8defender,1.1.15300.6,4.18.1809.2,1.277.230.0,0,7.0,0,NaN,49480.0,...,556.0,63396.0,1.0,NaN,0.0,0,0,0.0,1.0,11.0
3,0000055553dc51b1295785415f1a224d,win8defender,1.1.15400.5,4.18.1810.5,1.281.664.0,0,7.0,0,NaN,42160.0,...,628.0,26320.0,1.0,0.0,0.0,0,0,0.0,0.0,10.0
4,00000574cefffeca83ec8adf9285b2bf,win8defender,1.1.15400.4,4.18.1809.2,1.279.236.0,0,7.0,0,NaN,53447.0,...,556.0,63269.0,1.0,NaN,0.0,0,0,0.0,1.0,3.0


SALVANDO A COLUNA COM O ID

In [41]:
sub_df = pd.DataFrame({"MachineIdentifier": df_test["MachineIdentifier"]})
sub_df.head()

,MachineIdentifier
0,0000010489e3af074adeac69c53e555e
1,00000176ac758d54827acd545b6315a5
2,0000019dcefc128c2d4387c1273dae1d
3,0000055553dc51b1295785415f1a224d
4,00000574cefffeca83ec8adf9285b2bf


Excluindo colunas

In [42]:
df_test.drop(columns=['IsBeta','RtpStateBitfield','IsSxsPassiveMode','DefaultBrowsersIdentifier','AVProductStatesIdentifier',
                       'AVProductsInstalled','AVProductsEnabled','HasTpm','CountryIdentifier','CityIdentifier',
                       'OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier','OsBuild','OsSuite',
                       'IsProtected','AutoSampleOptIn','SMode','IeVerIdentifier','Firewall','UacLuaenable',
                       'Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorCoreCount',
                       'Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier','Census_PrimaryDiskTotalCapacity',
                       'Census_SystemVolumeTotalCapacity','Census_HasOpticalDiskDrive','Census_TotalPhysicalRAM','Census_InternalPrimaryDiagonalDisplaySizeInInches',
                       'Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical',
                       'Census_InternalBatteryNumberOfCharges','Census_OSBuildNumber','Census_OSBuildRevision','Census_OSInstallLanguageIdentifier',
                       'Census_OSUILocaleIdentifier','Census_IsPortableOperatingSystem','Census_IsFlightingInternal','Census_IsFlightsDisabled',
                       'Census_ThresholdOptIn','Census_FirmwareManufacturerIdentifier','Census_FirmwareVersionIdentifier','Census_IsSecureBootEnabled',
                       'Census_IsWIMBootEnabled','Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable',
                       'Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Wdft_RegionIdentifier','PuaMode','SmartScreen',
                       'Census_ProcessorClass','Census_PrimaryDiskTypeName','Census_ChassisTypeName','Census_PowerPlatformRoleName',
                       'Census_InternalBatteryType','OsBuildLab', 'MachineIdentifier'], axis = 1,inplace = True)

In [43]:
obj = (df_test.dtypes == 'category') #quais colunas que são categóricas dentro do dataset?
object_cols = list(obj[obj].index)

object_cols

['ProductName',
 'EngineVersion',
 'AppVersion',
 'AvSigVersion',
 'Platform',
 'Processor',
 'OsVer',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_MDC2FormFactor',
 'Census_DeviceFamily',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'Census_FlightRing']

In [44]:
encoder = OrdinalEncoder() #faça a troca das categorias por números

df_test[object_cols] = encoder.fit_transform(df_test[object_cols]) #ltere somente as colunas categóricas do meu dataset
df_test

,ProductName,EngineVersion,AppVersion,AvSigVersion,Platform,Processor,OsVer,OsPlatformSubRelease,SkuEdition,Census_MDC2FormFactor,...,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
0,4.0,68.0,70.0,9261.0,0.0,1.0,0.0,2.0,4.0,8.0,...,255.0,0.0,3.0,4.0,2.0,5.0,0.0,1.0,0.0,7.0
1,4.0,67.0,64.0,9005.0,0.0,1.0,0.0,3.0,4.0,8.0,...,323.0,0.0,6.0,4.0,2.0,6.0,0.0,1.0,2.0,7.0
2,4.0,65.0,64.0,8501.0,0.0,1.0,0.0,1.0,4.0,8.0,...,174.0,0.0,0.0,7.0,5.0,3.0,5.0,1.0,0.0,7.0
3,4.0,68.0,70.0,9301.0,0.0,1.0,0.0,3.0,4.0,8.0,...,312.0,0.0,5.0,7.0,5.0,8.0,3.0,1.0,0.0,7.0
4,4.0,67.0,64.0,8989.0,0.0,1.0,0.0,3.0,4.0,8.0,...,312.0,0.0,5.0,7.0,5.0,7.0,5.0,1.0,2.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7853248,4.0,68.0,70.0,9205.0,0.0,1.0,0.0,3.0,4.0,8.0,...,301.0,0.0,5.0,7.0,5.0,2.0,5.0,1.0,2.0,7.0
7853249,4.0,68.0,70.0,9226.0,0.0,1.0,0.0,4.0,6.0,8.0,...,370.0,0.0,8.0,20.0,17.0,2.0,2.0,1.0,4.0,7.0
7853250,4.0,68.0,70.0,9205.0,0.0,1.0,0.0,4.0,4.0,8.0,...,367.0,0.0,8.0,7.0,5.0,2.0,2.0,1.0,4.0,7.0
7853251,4.0,67.0,89.0,8881.0,0.0,2.0,0.0,5.0,6.0,8.0,...,39.0,2.0,20.0,20.0,17.0,2.0,5.0,0.0,4.0,4.0


SUBMISSÃO LightGBM.

Pontuação: 49,6

SUBMISSÃO Extreme Gradient Boosting

Pontuação: 49.18

In [45]:
predicao_test = lgb_dart.predict(df_test)
predicao_test

array([0., 0., 0., ..., 0., 0., 0.], dtype=float16)

In [46]:
predictions_probas = lgb_dart.predict_proba(df_test)
predictions_probas

array([[0.92092684, 0.07907316],
       [0.92697756, 0.07302244],
       [0.92023063, 0.07976937],
       ...,
       [0.92633037, 0.07366963],
       [0.86406575, 0.13593425],
       [0.92633037, 0.07366963]])

CSV SUBMISSAO

In [47]:
pred_final = pd.DataFrame(predictions_probas)[1]

In [48]:
#subm_df = pd.read_csv('../input/microsoft-malware-prediction/sample_submission.csv', index_col='MachineIdentifier')
#subm_df['HasDetections'] = predictions_probas

#subm_df.head()

In [49]:
sub_df["HasDetections"] = pred_final
sub_df[:10]

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.079073
1,00000176ac758d54827acd545b6315a5,0.073022
2,0000019dcefc128c2d4387c1273dae1d,0.079769
3,0000055553dc51b1295785415f1a224d,0.079769
4,00000574cefffeca83ec8adf9285b2bf,0.073670
5,000007ffedd31948f08e6c16da31f6d1,0.073670
6,000008f31610018d898e5f315cdf1bd1,0.079769
7,00000a3c447250626dbcc628c9cbc460,0.073670
8,00000b6bf217ec9aef0f68d5c6705897,0.092801
9,00000b8d3776b13e93ad83676a28e4aa,0.073670


In [50]:
sub_df.to_csv('/kaggle/working/submission.csv', index=False)